In [229]:
import random
from bitstring import BitArray

In [230]:
#python script to create test cases
def add(a, b):
    return bin((int(a, 2) + int(b, 2))).replace("0b", "")[-32:]

def sub(a, b):
    bs = bin(BitArray(bin=a).int-BitArray(bin=b).int).replace("0b", "")
    fb = bs[0]

    if (fb == "-"):
        return "1" + (bs[-31:].replace("-", "").rjust(31, "0"))
    else:
        return "0" + (bs[-31:].rjust(31, "0"))

def and_op(a, b):
    return "".join(["1" if (x == y and x != "0" and y != "0") else "0" for x, y in zip(a, b)])
    
def or_op(a, b):
    return "".join(["1" if (x == "1" or y == "1") else "0" for x, y in zip(a, b)])

def xor_op(a, b):
    return "".join(["1" if (x != y) else "0" for x, y in zip(a, b)])

def nor_op(a, b):
    return "".join(["1" if (c == "0") else "0" for c in or_op(a, b)])
    
def slt_op(a, b):
    return "1".rjust(32, "0") if BitArray(bin=a).int < BitArray(bin=b).int else "0".rjust(32, "0") 

def hexify(a):
    return hex(int(a, 2)).replace("0x", "").rjust(8, "0")

    
class Verifier:
    def __init__(self, width=32):
        self.width = width

    def verify(self,
               a : str,
               b : str,
               op : int) -> str:
        match op:
            # add
            case 0: return add(a, b)
            case 2: return sub(a, b)
            case 4: return and_op(a, b)
            case 5: return or_op(a, b)
            case 6: return xor_op(a, b)
            case 7: return nor_op(a, b)
            case 10: return slt_op(a, b)

        raise Exception("no Operation mapped")

class Generator:
    def __init__(self, width=32, seed=0, verifier=Verifier()):
        self.width = 32
        self.v = verifier
        random.seed(seed)

    def generate(self):
        return "".join(random.choice(["0", "1"]) for i in range(0, self.width))

    def make_case(self) -> list[str]:
        a = self.generate()
        b = self.generate()
        opcode = random.choice([0, 2, 4, 5, 6, 7, 10])

        res = self.v.verify(a, b, opcode)
        return {"op": opcode, "a": a, "b": b, "res": res}

In [231]:
v = Verifier()
g = Generator()
a = g.generate()
b = g.generate()
hexify(a)

'df929bb8'

In [233]:
print("a   "+a)
print("b   "+b)
print("add " + v.verify(a, b, 0))
print("sub " + v.verify(a, b, 2))
print("slt " + v.verify(a, b, 10))
print("and " + v.verify(a, b, 4))
print("or  " + v.verify(a, b, 5))
print("nor " + v.verify(a, b, 7))
print("xor " + v.verify(a, b, 6))

a   11011111100100101001101110111000
b   10110100000100110110101101101000
add 10010011101001100000011100100000
sub 00101011011111110011000001010000
slt 00000000000000000000000000000000
and 10010100000100100000101100101000
or  11111111100100111111101111111000
nor 00000000011011000000010000000111
xor 01101011100000011111000011010000


In [238]:
N = 50

with open("test.txt", "w") as f:
    for i in range(0, N):
        d = g.make_case()
        f.write(f"{hex(d['op'])[2:4].upper()}_______{hexify(d['a']).upper()}___{hexify(d['b']).upper()}___{hexify(d['res']).upper()}\n")
        

In [206]:
BitArray(bin="01").int

1

In [200]:
BitArray(hex="97016717")

BitArray('0x97016717')